In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model='gpt-4o-mini')
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '당신은 금융 상담사입니다. 사용자에게 최선의 금융 조언을 제공합니다.'),
        ('placeholder', '{messages}'),
    ]
)

In [2]:
chain = prompt | chat

In [4]:
ai_msg = chain.invoke(
    {
        'messages': [
            ('human', '저축을 늘리기 위해 무엇을 할 수 있나요?'),
            ('ai', '저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요.'),
            ('human', '방금 뭐라고 했나요?'),
        ],
    }
)
print(ai_msg.content)

저축을 늘리기 위해, 저축 목표를 설정하고 매달 자동 이체로 일정 금액을 저축하는 방법을 추천했습니다. 이렇게 하면 규칙적으로 저축할 수 있어 더 효과적으로 저축을 늘릴 수 있습니다. 추가로, 소비 습관을 점검하고 불필요한 지출을 줄이는 것도 도움이 됩니다.


In [5]:
# ChatMessagehistory

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
chat_history = ChatMessageHistory()
chat_history.add_user_message('저축을 늘리기 위해 무엇을 할 수 있나요?')
chat_history.add_ai_message('저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요.')

In [9]:
chat_history.add_user_message('방금 뭐라고 했나요?')
ai_response = chain.invoke({'messages':chat_history.messages})
print(ai_response.content)

저축을 늘리기 위해 저축 목표를 설정하고, 매달 자동 이체를 통해 일정 금액을 저축하는 것이 좋다고 말씀드렸습니다. 이렇게 하면 저축을 보다 효율적으로 관리할 수 있습니다. 추가적으로 도움이 필요하시면 말씀해 주세요!


In [10]:
# RunnablewithMessageHistory

In [11]:
from langchain_core.runnables.history import RunnableWithMessageHistory

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '당신은 금융 상담사입니다. 모든 질문에 최선을 다해 답변하십시오.'),
        ('placeholder', '{chat_history}'),
        ('human', '{input}'),
    ]
)

In [13]:
chat_history = ChatMessageHistory()
chain = prompt|chat

In [14]:
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key='input',
    history_messages_key='chat_history',
)

In [15]:
chain_with_message_history.invoke(
    {'input':'저축을 늘리기 위해 무엇을 할 수 있나요?'},
    {'configurable':{'session_id':'unused'}},
).content

'저축을 늘리기 위해 다음과 같은 여러 가지 방법을 고려할 수 있습니다:\n\n1. **예산 수립**: 매달 수입과 지출을 정리하여 어디에 돈이 얼마나 쓰이는지 파악하세요. 불필요한 지출을 줄이는 데 도움이 됩니다.\n\n2. **정기 저축**: 월급이 들어올 때 일정 금액을 자동으로 저축 계좌로 이체되도록 설정하세요. 이렇게 하면 저축이 더 수월해집니다.\n\n3. **비상금 마련**: 예기치 않은 지출에 대비해 비상금을 만들어두세요. 일반적으로 3~6개월치 생활비 정도가 적당합니다.\n\n4. **소비 패턴 분석**: 주기적으로 소비 패턴을 검토하고 불필요한 지출을 줄이세요. 예를 들어, 식음료 비용이나 구독 서비스를 점검해볼 수 있습니다.\n\n5. **상여금 및 보너스 활용**: 예상치 못한 추가 수입이 들어온다면, 이를 저축하거나 투자하는 데 사용해보세요.\n\n6. **할인 및 쿠폰 활용**: 쇼핑할 때 할인 쿠폰이나 세일 정보를 활용하여 비용을 절감하고, 절약된 금액을 저축하세요.\n\n7. **삶의 우선순위 재조정**: 현재의 라이프스타일을 검토하고 필요 없는 지출을 줄여 저축액을 늘려 보세요.\n\n8. **재무 목표 설정**: 저축의 목적을 명확하게 정리하고, 이를 위한 목표 금액과 기간을 설정하세요. 이는 동기를 부여하는 데 큰 도움이 됩니다.\n\n9. **금융 상품 활용**: 고금리 저축 상품이나 적금, 펀드 등의 금융 상품을 활용해 보세요. 그러나 리스크를 잘 따져보고 선택하세요.\n\n10. **주기적인 검토**: 정기적으로 저축 현황을 검토하고 필요한 경우 조정하세요. 목표에 도달하기 위해 어떤 추가적인 조치가 필요한지를 고민해보세요.\n\n위의 방법들을 통해 저축을 효과적으로 늘릴 수 있을 것입니다. 재무 목표를 향해 한 걸음 한 걸음 나아가길 바랍니다!'

In [16]:
chain_with_message_history.invoke(
    {'input':'내가 방금 뭐라고 했나요?'},
    {'configurable':{'session_id':'unused'}},
).content

'당신은 "저축을 늘리기 위해 무엇을 할 수 있나요?"라고 질문하셨습니다. 이 질문에 대해 여러 가지 방법을 제안했습니다. 추가적인 질문이나 더 알고 싶은 내용이 있으시면 말씀해 주세요!'

In [17]:
# Message Trimming

In [19]:
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
trimmer = trim_messages(strategy='last', max_tokens=2, token_counter=len)

In [21]:
chain_with_trimming = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history')|trimmer)
    | prompt
    | chat
)

In [22]:
chain_with_trimmed_history = RunnableWithMessageHistory(
    chain_with_trimming,
    lambda session_id: chat_history,
    input_messages_key='input',
    history_messages_key='chat_history',
)

In [23]:
chain_with_trimmed_history.invoke(
    {'input':'저는 5년 내에 집을 사기 위해 어떤 재정 계획을 세워야 하나요?'},
    {'configurable':{'session_id':'finance_session_1'}},
)

AIMessage(content='5년 내에 집을 사기 위한 재정 계획을 세우는 것은 매우 중요한 단계입니다. 다음 단계들을 고려해보세요:\n\n1. **목표 설정**:\n   - 집을 구매할 지역과 예산을 설정하세요. 이 지역의 평균 집값을 조사해보고, 얼마를 모아야 할지 계산하세요.\n\n2. **예산 작성**:\n   - 현재 수입과 지출을 분석하여 월별로 얼마를 저축할 수 있는지 파악하세요. 불필요한 지출을 줄여 저축할 여력을 늘리는 것이 중요합니다.\n\n3. **저축 목표**:\n   - 집 가격의 20%를 다운페이로 삼는 것이 일반적입니다. 목표 금액에 도달하기 위해 매달 저축해야 할 금액을 계산하세요.\n\n4. **저축 계좌 개설**:\n   - 고이율 저축 계좌나 적금 상품을 찾아 해당 자산을 저축하세요. 일부 국가에서는 주택 구매를 위한 세금 우대 계좌가 제공되기도 합니다.\n\n5. **신용 점수 관리**:\n   - 높은 신용 점수는 대출을 받을 때 유리합니다. 자신의 신용 점수를 확인하고 개선이 필요한 부분을 해결하세요.\n\n6. **대출 옵션 조사**:\n   - 여러 금융기관의 모기지 옵션을 비교하고, 이자율, 대출 가능한 금액, 상환 기간 등을 조사하세요.\n\n7. **부가 수입 고려**:\n   - 추가적인 수입을 얻을 수 있는 방법을 고려해 보세요. 파트타임 일이나 투자 수익 등을 통해 저축액을 더 늘릴 수 있습니다.\n\n8. **전문가 상담**:\n   - 필요시 재정 상담사와 상담하여 개인화된 조언을 받을 수도 있습니다. \n\n이 단계들을 잘 따르고 계획적으로 진행하면, 5년 후 집을 구매하는 목표를 달성하는 데 큰 도움이 될 것입니다. 추가적인 질문이 있으시면 언제든지 물어보세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 435, 'prompt_tokens': 121, 'total_tok

In [24]:
chain_with_trimmed_history.invoke(
    {'input':'내가 방금 뭐라고 했나요?'},
    {'configurable':{'session_id':'finance_session_1'}},
).content

'당신은 "저는 5년 내에 집을 사기 위해 어떤 재정 계획을 세워야 하나요?"라고 질문하셨습니다. 이에 대한 답변으로 집 구매를 위한 재정 계획에 대한 여러 가지 단계를 설명드렸습니다. 추가적인 질문이나 더 구체적인 정보가 필요하시면 말씀해 주세요!'

In [27]:
def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages)==0:
        return False

    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ('placeholder', '{chat_history}'),
            ('user', '이전 대화를 요약해 주세요. 가능한 한 많은 세부 정보를 포함하십시오.'),
        ]
    )
    summarization_chain = summarization_prompt | chat
    summary_message = summarization_chain.invoke({'chat_history':stored_messages})

    chat_history.clear()
    chat_history.add_message(summary_message)

    return True

In [28]:
chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

In [30]:
print(chain_with_summarization.invoke(
    {'input':'저에게 어떤 재정적 조언을 해주셨나요?'},
    {'configurable':{'session_id':'unused'}},
).content)

제가 이전에 드린 재정적 조언은 다음과 같습니다:

1. **저축 늘리기**:
   - **예산 수립**: 수입과 지출을 정리하여 불필요한 지출을 줄이고 저축할 수 있는 금액을 파악.
   - **정기 저축**: 자동 이체를 통해 매달 일정 금액을 저축 계좌로 이체.
   - **비상금 마련**: 3~6개월치 생활비를 비상금으로 확보하여 예기치 않은 상황에 대비.
   - **소비 패턴 분석**: 지출을 검토하고 불필요한 항목을 줄이기.
   - **상여금 및 보너스 활용**: 추가 수입이 들어올 때 이를 저축에 활용하기.
   - **할인 및 쿠폰 활용**: 쇼핑 시 할인과 쿠폰을 이용해 절약한 돈을 저축.
   - **재무 목표 설정**: 구체적인 저축 목표(예: 여행, 집 구매 등) 설정.
   - **금융 상품 활용**: 고금리 저축 계좌나 적금을 고려하여 돈을 효율적으로 운용.
   - **주기적인 검토**: 저축 및 재무 목표를 정기적으로 점검하고 필요 시 조정.

2. **5년 내 집 구매를 위한 재정 계획**:
   - **목표 설정**: 구매할 집의 지역과 예산을 결정하고 평균 집값 조사.
   - **예산 작성**: 월별 수입 및 지출을 분석하여 저축 가능한 금액을 계산.
   - **저축 목표**: 집 가격의 20% 정도를 다운페이로 설정하고 필요한 저축 금액 계산.
   - **저축 계좌 개설**: 고이율 저축 계좌나 적금 상품 활용하여 저축 극대화.
   - **신용 점수 관리**: 신용 점수를 확인하고 필요 시 점수를 개선하기 위한 방법 모색.
   - **대출 옵션 조사**: 다양한 금융기관의 모기지(주택 담보 대출) 옵션을 비교.
   - **부가 수입 고려**: 불법한 방법이 아닌, 추가적인 수입원을 찾아보는 방법.
   - **전문가 상담**: 재무 상담사 또는 부동산 전문가와 상담하여 개인화된 조언을 받는 것.

이 조언들은 저축을 늘리고 미래의 재정적 목표(특히 주택 구매)를 달성하는 데 도움이 될 수 있습니다. 추가적인 궁